In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

In [2]:
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0301, -0.1732, -0.0337,  0.0214,  0.0420, -0.0324, -0.0536,  0.1165,
         -0.2211,  0.1709],
        [ 0.0595, -0.0213,  0.0153,  0.0218,  0.0729, -0.1081,  0.0933, -0.0291,
         -0.0629,  0.1520]], grad_fn=<AddmmBackward0>)

In [5]:
class MLP(nn.Module):
    # 用模型参数声明层，这里，我们声明两个全连接的层
    def __init__(self):
        # 调用 `MLP` 的父类 `Block` 的构造函数来执行必要的初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数 `params`
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    # 定义模型的正向传播，即如何根据输入 `X` 返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用 ReLU 的函数版本，其在 nn.functional 模块中定义
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[-0.0408, -0.1128,  0.0127,  0.1328, -0.0829, -0.0873, -0.2309, -0.0496,
          0.0683, -0.0851],
        [-0.1126, -0.0654, -0.1791,  0.0674, -0.0897, -0.0743, -0.2763, -0.1068,
          0.1247, -0.0879]], grad_fn=<AddmmBackward0>)

In [18]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 这里，`block` 是 `Module` 子类的一个实例。我们把它保存在 `Module` 类的成员变量
            # `_modules` 中。`block` 的类型是 OrderedDict
            self._modules[block] = block
        
    def forward(self, X):
        # OrderedDict 保证了按照成员添加的顺序遍历它们
        for block in self._modules.values(): # self._modules 好像也行，因为键值一样
            X = block(X)
        return X

In [19]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1804, -0.0425,  0.1637, -0.0507, -0.2202, -0.0895,  0.0762, -0.2013,
          0.1918,  0.3458],
        [-0.0870, -0.1006,  0.1480, -0.0243, -0.0688, -0.1911, -0.0972, -0.1489,
          0.2505,  0.3743]], grad_fn=<AddmmBackward0>)

In [20]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重函数，因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及 `relu` 和 `dot` 函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [21]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1398, grad_fn=<SumBackward0>)

In [22]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0286, grad_fn=<SumBackward0>)

In [30]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0826],
        [0.1102]], grad_fn=<AddmmBackward0>)

In [31]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2977, -0.0771,  0.1549, -0.2106, -0.2900, -0.1914, -0.2428,  0.2992]])), ('bias', tensor([0.1465]))])


In [32]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1465], requires_grad=True)
tensor([0.1465])


In [34]:
net[2].weight.grad == None

True

In [40]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [42]:
net.state_dict()['2.bias'].data, net[0].state_dict()['weight'].data

(tensor([0.1465]),
 tensor([[-0.2505,  0.2664, -0.4775, -0.0965],
         [-0.2245,  0.3516, -0.0359, -0.3562],
         [-0.0671, -0.2525, -0.1833,  0.1296],
         [ 0.0057,  0.1628, -0.2042, -0.0556],
         [-0.4654,  0.3932,  0.1064, -0.3155],
         [-0.3419, -0.2289, -0.4666, -0.3685],
         [-0.3559, -0.2365,  0.3638,  0.1206],
         [-0.2437, -0.4389, -0.4058, -0.4598]]))

In [44]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.5142],
        [-0.5143]], grad_fn=<AddmmBackward0>)

In [45]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [46]:
rgnet[0][1][0].bias.data

tensor([ 0.3922,  0.3450,  0.4158, -0.1827, -0.3563, -0.3148,  0.4340, -0.4607])

In [47]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0016, -0.0128,  0.0047, -0.0153]), tensor(0.))

In [49]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [50]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6933,  0.1758,  0.6927,  0.5400])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [55]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0]) # [0] 是指定 weight，[1] 是 bias
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5 # (-5, 5) 之间的设为 0
    
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.7020,  9.6512,  0.0000, -0.0000],
        [ 7.8431, -0.0000, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [56]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[4].weight.data[0, 0])

tensor([True, True, True, True, True, True, True, True])
tensor(100.)


In [57]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [58]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [59]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-7.4506e-09, grad_fn=<MeanBackward0>)

In [60]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [61]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 1.4761, -0.9188,  1.1741],
        [ 2.4247, -0.6346, -0.1589],
        [ 0.4079,  0.2278,  0.0213],
        [-1.9083, -0.8743, -0.0706],
        [ 1.5651, -1.0459, -1.1265]], requires_grad=True)

In [62]:
linear(torch.rand(2, 5))

tensor([[3.5052, 0.0000, 0.7334],
        [3.2991, 0.0000, 0.0000]])

In [64]:
net = nn.Sequential(MyLinear(64, 8), CenteredLayer(), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[18.9373],
        [ 4.8549]])

In [65]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [66]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [67]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [68]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [70]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [71]:
torch.save(net.state_dict(), 'mlp.params')

In [72]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval() # 用来让模型“准备好”，以便下面的语句接收测试数据

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [73]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [79]:
import os
os.chdir('C:\\Program Files\\NVIDIA Corporation\\NVSMI')
!nvidia-smi

Fri Nov 19 17:09:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.19       Driver Version: 472.19       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   46C    P8    N/A /  N/A |    855MiB /  4096MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [80]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [torch.device(f'cuda:{i}')
              for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [81]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [83]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [87]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu()) # 直接 net.to(device=try_gpu()) 也可以，.to() 是 in-place 操作

2277289330480
2277289330480


In [85]:
net(X)

tensor([[0.4430],
        [0.4430]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [86]:
net[0].weight.data.device

device(type='cuda', index=0)